In [29]:
import pandas as pd
import numpy as np
from nsepython import equity_history
from datetime import datetime, timedelta

In [52]:
symbol = "INFY"

def get_historical_stock_price(symbol, to_date = datetime.today().strftime("%d-%m-%Y"), 
                                   from_date = (datetime.today() - timedelta(days = 367)).strftime("%d-%m-%Y")):
    df = equity_history(symbol, "EQ", from_date, to_date)
    df = df.drop(columns = ['_id','CH_SYMBOL','CH_SERIES','CH_MARKET_TYPE', 'CH_LAST_TRADED_PRICE',
                                    'CH_PREVIOUS_CLS_PRICE','CH_TOTAL_TRADES','TIMESTAMP','createdAt',
                                    'updatedAt','__v','VWAP','mTIMESTAMP','CA'])
    df.columns = ['High', 'Low', 'Open', 'Close', 'Total Traded Quantity', 'Total Traded Value', 
                  '52 week High', '52 week Close', 'ISIN', 'Date']
    return df.set_index('Date')

In [53]:
data = get_historical_stock_price(symbol)

In [54]:
data['candle_body_length'] = (data['Close'] - data['Open']).abs()
data['candle_length'] = data['High'] - data['Low']
data['candle_body_ratio'] = (data['candle_body_length'] * 100 / data['candle_length'])
data['day_change_perc'] = data['candle_body_length'] * 100 / data['Close'].shift(1)
data['candle_color'] = np.where(data['Open'] > data['Close'], "Red", np.where(data['Open'] != data['Close'], "Green", "")) 
data['upper_wick_length'] = data['High'] - data[['Close','Open']].max(axis = 1)
data['lower_wick_length'] = data[['Close','Open']].min(axis = 1) - data['Low']

In [56]:
def marubozu(data):
    return pd.Series(np.where(np.logical_and(data['candle_body_ratio'] > 86.5, data['day_change_perc'] > 1.25), 
                                      data['candle_color'], "")).replace({'Red' : 'Bearish', 'Green' : 'Bullish'}).tolist()

def spinning_top_doji(data):
    wick_conditions = np.logical_and((data['upper_wick_length'] / data['candle_length']).between(.25, .75), (data['lower_wick_length'] / data['candle_length']).between(.25, .75))
    body_condition = np.where(data['candle_body_ratio'] <= 17.5, "Doji", np.where(data['candle_body_ratio'] <= 35, "Spinning Top", None))
    return np.where(np.logical_and(body_condition, wick_conditions), body_condition, "")

In [57]:
data['marubozu'] = marubozu(data)

In [58]:
data['spinning_top'] = spinning_top_doji(data)

In [59]:
# paper umbrella
data[np.logical_and(data['candle_body_ratio'] >= 10, np.logical_and((data['lower_wick_length'] / data['candle_length'] > .65),data['lower_wick_length'] >= (1.88 * data['candle_body_length'])))]

,High,Low,Open,Close,Total Traded Quantity,Total Traded Value,52 week High,52 week Close,ISIN,candle_body_length,candle_length,candle_body_ratio,day_change_perc,candle_color,upper_wick_length,lower_wick_length,marubozu,spinning_top
Date,,,,,,,,,,,,,,,,,,
2022-08-02,1547.00,1528.35,1546.55,1543.60,4374635,6.728992e+09,1953.90,1367.15,INE009A01021,2.95,18.65,15.817694,0.190194,Red,0.45,15.25,,
2022-06-17,1396.95,1367.15,1391.05,1387.30,9692047,1.341970e+10,1953.90,1367.15,INE009A01021,3.75,29.80,12.583893,0.268394,Red,5.90,20.15,,
2022-01-13,1912.50,1866.35,1905.00,1896.80,14277630,2.704461e+10,1914.05,1231.00,INE009A01021,8.20,46.15,17.768147,0.436763,Red,7.50,30.45,,
2022-02-01,1780.00,1734.70,1766.10,1772.05,5119935,9.046788e+09,1953.90,1241.00,INE009A01021,5.95,45.30,13.134658,0.342702,Green,7.95,31.40,,
2021-11-08,1741.00,1703.00,1730.00,1737.50,5245903,9.063983e+09,1848.00,1078.00,INE009A01021,7.50,38.00,19.736842,0.438930,Green,3.50,27.00,,
